In [ ]:
import requests
from lsst.rsp.utils import get_access_token
import time

In [ ]:
username = os.getenv('USER')

In [ ]:
url = f"https://data-int.lsst.cloud/files/{username}/home/{username}"
base_files_url = f"https://data-int.lsst.cloud/files/"

timestamp = int(time.time())
filename = f"test_{timestamp}.txt"
test_content = "Testing WebDAV"

In [ ]:
# Auth stuff
token=get_access_token()
headers = {'Authorization': f'Bearer {token}'}

In [ ]:
# GET to /files endpoint to initialize
print("Getting /files endpoint...")

response = requests.get(base_files_url, headers=headers)

if response.status_code != 200:
    raise Exception(f"/files endpoint failed!: {response.status_code} - {response.text}")

In [ ]:
# Do a PUT of the file to WebDAV
print("Writing file")

response = requests.put(f"{url}/{filename}", 
                       data=test_content, 
                       headers=headers)

if response.status_code not in [200, 201, 204]:
    raise Exception(f"File upload failed: {response.status_code} - {response.text}")

In [ ]:
# List files with PROPFIND
print("Listing files")

response = requests.request('PROPFIND', f"{url}/", 
                           headers={**headers, 'Depth': '1'})

print(f"Status: {response.status_code}")

if response.status_code != 207:
    raise Exception(f"Directory listing failed: {response.status_code} - {response.text}")

if filename not in response.text:
    raise Exception(f"File {filename} not found in directory listing")

In [ ]:
# Download file and check content
print("Downloading file")

response = requests.get(f"{url}/{filename}", headers=headers)
print(f"Status: {response.status_code}")

if response.status_code != 200:
    raise Exception(f"File download failure: {response.status_code} - {response.text}")

downloaded_content = response.text
print(f"Downloaded content: '{downloaded_content}'")

if downloaded_content != test_content:
    raise Exception(f"Content validation Failed - Expected: '{test_content}' Got: '{downloaded_content}'")

print("File downloaded and content is as expected")

In [ ]:
# Delete file from WebDAV
print("Deleting file")

response = requests.delete(f"{url}/{filename}", headers=headers)
print(f"Status: {response.status_code}")

if response.status_code not in [200, 204, 404]:
    raise Exception(f"File deletion failed: {response.status_code} - {response.text}")

In [ ]:
# Validate that it was deleted
print("Checking if deleted")

response = requests.request('PROPFIND', f"{url}/", 
                           headers={**headers, 'Depth': '1'})

if filename not in response.text:
    print(f"{filename} deleted")
else:
    print(f"{filename} still exists")

if response.status_code != 207:
    raise Exception(f"Verification listing failed! {response.status_code} - {response.text}")

if filename in response.text:
    raise Exception(f"File {filename} was not correctly deleted!")